In [2]:
!pip install ktrain

     |████████████████████████████████| 174kB 3.4MB/s 
     |████████████████████████████████| 317kB 44.8MB/s 
     |████████████████████████████████| 112kB 49.6MB/s 
     |████████████████████████████████| 1.0MB 34.5MB/s 
     |████████████████████████████████| 204kB 47.5MB/s 
     |████████████████████████████████| 1.8MB 41.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.5.2-cp36-none-any.whl size=213552 sha256=f46a4f7be1c580750284b89e4f189d282e2e245cf2e604e5713fa28d0d76df3a
  Stored in directory: /root/.cache/pip/wheels/72/71/d8/54ea825a57a816701162b9b0696765593182ce37d9f30d7819
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=631e82c3029f97544da45fe326c28f88a425fce3a848c100e59b39b58b78dc4d
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=9d9c63ce4723f128b9e56c57c122977ff247326dab289c14c0f

In [3]:
import ktrain
from ktrain import vision as vis

Using TensorFlow backend.


using Keras version: 2.2.4


In [0]:
DATADIR = '/content/Chest-X-Ray-Images-Pneumonia'

In [11]:
(train_data,val_data,preproc)=vis.images_from_folder(datadir=DATADIR,data_aug = vis.get_data_aug(horizontal_flip=True),train_test_names=['train','test'],target_size=(224,224),color_mode='rgb')

color_mode detected (grayscale) different than color_mode selected (rgb)
Found 5216 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [13]:
model = vis.image_classifier('pretrained_resnet50', train_data, val_data, multilabel=True)

The normalization scheme has been changed for use with a pretrained_resnet50 model. If you decide to use a different model, please reload your dataset with a ktrain.vision.data.images_from* function.

Is Multi-Label? True
pretrained_resnet50 model created.


In [0]:
learner = ktrain.get_learner(model=model, train_data=train_data, val_data=val_data, 
                             workers=8, use_multiprocessing=False, batch_size=64)

In [17]:
learner.fit_onecycle(0.0001, 1)



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/1
82/82 [==============================] - 1631s 20s/step - loss: 0.2776 - acc: 0.9389 - val_loss: 1.2823 - val_acc: 0.7652


In [18]:
loss, acc = learner.model.evaluate_generator(val_data, steps=len(val_data))
print('final loss:%s, final score:%s' % (loss, acc))

final loss:1.2822551812069156, final score:0.7652243574460348


In [19]:
!!ls /content/Chest-X-Ray-Images-Pneumonia/test/PNEUMONIA |head -n3

['person100_bacteria_475.jpeg',
 'person100_bacteria_477.jpeg',
 'person100_bacteria_478.jpeg']

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)


In [25]:
predictor.predict_filename('/content/Chest-X-Ray-Images-Pneumonia/test/PNEUMONIA/person147_bacteria_707.jpeg')

[[('NORMAL', 0.0), ('PNEUMONIA', 1.0)]]